In [1]:
import numpy as np
import pandas as pd
import pydicom
%matplotlib inline
import matplotlib.pyplot as plt

from tensorflow.keras.models import model_from_json, load_model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout, Flatten, Conv2D, MaxPooling2D, BatchNormalization
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.resnet import ResNet50 
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, LearningRateScheduler, EarlyStopping, ReduceLROnPlateau

from skimage.transform import rescale, resize, downscale_local_mean

In [2]:
def load_pretrained_model():
    
    model = VGG16(include_top=False, weights='imagenet', input_shape = (512, 512, 3))
    
    return model

def build_my_model():
    
    # my_model = Sequential()
    # ....add your pre-trained model, and then whatever additional layers you think you might
    # want for fine-tuning (Flatteen, Dense, Dropout, etc.)
    
    # if you want to compile your model within this function, consider which layers of your pre-trained model, 
    # you want to freeze before you compile 
    
    # also make sure you set your optimizer, loss function, and metrics to monitor
    
    vgg_model = load_pretrained_model()
    
    for layer in vgg_model.layers[:17]:
        layer.trainable = False
        
    my_model = Sequential()
    
    my_model.add(vgg_model)
    
    my_model.add(GlobalAveragePooling2D())
    
    my_model.add(Dense(256, activation='relu'))
    
    my_model.add(BatchNormalization())
    
    my_model.add(Dropout(0.4))
    
    my_model.add(Dense(64, activation='relu'))
    
    my_model.add(BatchNormalization())
    
    my_model.add(Dropout(0.3))
    
    my_model.add(Dense(32, activation='relu'))
    
    my_model.add(BatchNormalization())
    
    my_model.add(Dropout(0.3))
    
    my_model.add(Dense(1, activation='sigmoid'))
    
    return my_model

In [3]:
# This function reads in a .dcm file, checks the important fields for our device, and returns a numpy array
# of just the imaging data
def check_dicom(filename): 
    
    print('Load file {} ...'.format(filename))
    ds = pydicom.dcmread(filename)   
    
    if(ds.Modality == 'DX' and ds.PatientPosition in ['PA', 'AP'] and (ds.BodyPartExamined == 'CHEST')):
        img = ds.pixel_array
        return img
    
    print('Invalid data...')
    return None
    
# This function takes the numpy array output by check_dicom and 
# runs the appropriate pre-processing needed for our model input
def preprocess_image(img, img_size): 
    
    img = resize(img, (img_size[1], img_size[2]))
    
    proc_img = (img - img.mean()) / img.std()
    
    return np.resize(proc_img, img_size)

# This function loads in our trained model w/ weights and compiles it 
def load_model(model_path):
    
    my_model = build_my_model()
    my_model.load_weights(model_path)
    
    return my_model

# This function uses our device's threshold parameters to predict whether or not
# the image shows the presence of pneumonia using our trained model
def predict_image(model, img, thresh): 
    
    prediction = model(img)
    
    if(prediction > thresh):
        prediction = 'Pneumonia'
    else:
        prediction = "No Pneumonia"
        
    return prediction 

In [4]:
test_dicoms = ['test1.dcm','test2.dcm','test3.dcm','test4.dcm','test5.dcm','test6.dcm']

model_path = r'xray_class_my_model.h5'

IMG_SIZE=(1,512,512,3) # This might be different if you did not use vgg16

my_model = load_model(model_path)

thresh = 0.25

# # use the .dcm files to test your prediction
for i in test_dicoms:
    
    img = check_dicom(i)
    
    if img is None:
        continue
    
    img_proc = preprocess_image(img, IMG_SIZE)

    pred = predict_image(my_model,img_proc,thresh)
    print(pred)

OSError: Unable to open file (unable to open file: name = 'xray_class_my_model.h5', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)